# 전처리

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
# hyperparameter settings
tolerance = 0.8 # micro-aggregation level에서 몇%의 잘못된 결과를 허용하는지 여부 (0.5 ~ 1.0)
categorize = 3 # 연속형 변수를 몇 개의 양자화를 허용할 것인지 여부 (2 ~ 3)

In [23]:
df1 = pd.read_csv("internal.csv", encoding = "utf-8-sig")
df2 = pd.read_csv("integrated.csv", encoding = "utf-8-sig")
df3 = pd.read_csv("mediator.csv", encoding = "utf-8-sig")

In [24]:
for i in tqdm(df2.columns):
    for w in df2[i]:
        try:
            if "기타" in w:
                print(w, " ", i)
                break
            elif "X" in w:
                print(w, " ", i)
                break
        except:
            pass

  5%|▍         | 3/65 [00:02<00:41,  1.50it/s]

기타   MGPO_OFFI


  8%|▊         | 5/65 [00:02<00:31,  1.91it/s]

04_기타   JOB
04_XX   INCOME


 40%|████      | 26/65 [00:18<00:32,  1.21it/s]

06.기타   TIME_SEG


 46%|████▌     | 30/65 [00:20<00:22,  1.56it/s]

기타   MAIN_AREA


100%|██████████| 65/65 [00:48<00:00,  1.34it/s]


In [25]:
# 제외 컬럼
del df2["PET_ACTIVE_SENIOR"]
del df2["OTHER_CNT"]
del df2["OTHER_AMT"]
del df2["OTHER_RATE"]
del df2["PER_OCH_WEDD"]

# 기타 및 XX 제외
df2["JOB"] = [np.nan if i == "04_기타" else i for i in df2["JOB"]]
df2["INCOME"] = [np.nan if i == "04_XX" else i for i in df2["INCOME"]]
df2["TIME_SEG"] = [np.nan if i == "06.기타" else i for i in df2["TIME_SEG"]]

# 서브 정보 활용
del df2["MGPO_MAIN"]
del df2["MGPO_HOM"]
del df2["SGG_HOM"]
del df2["MGPO_OFFI"]
del df2["SGG_OFFI_SEG"]
del df2["SGG_MAIN"]
del df2["DONG_MAIN"]
del df2["MAIN_AREA"]

# AMT CNT
del df2["OTHER_CARE_CNT"]
del df2["CMPPT_CNT"]
del df2["ONLINE_CSMTIC_CNT"]
del df2["DRGSTR_CNT"]
del df2["ONLINE_CNT"]
del df2["NUTRI_CNT"]

# >= 3만 해당 스테이지로 가정
df2["PER_WEDD_EXP"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_WEDD_EXP"])]
df2["PER_CHDB_EXP"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_CHDB_EXP"])]
# df2["PER_OCH_WEDD"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_OCH_WEDD"])]
df2["PER_HOUSEWF"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_HOUSEWF"])]
df2["PER_ONEPER"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_ONEPER"])]
df2["PER_PET"] = [1 if i >= 1 else 0 for i in tqdm(df2["PER_PET"])]

100%|██████████| 1309363/1309363 [00:00<00:00, 3534065.09it/s]


In [29]:
seg = dict(zip(df3["INCS_NO"], df3["SEG_KEY"]))
df1["seg_key"] = df1["INCS_NO"].progress_apply(lambda x : seg[x])

seg = dict(zip(df1["seg_key"], df1["BEHAVIOR_LABEL"]))
df2["BEHAVIOR_LABEL"] = df2["SEG_KEY"].progress_apply(lambda x : seg[x])

100%|██████████| 1309363/1309363 [00:01<00:00, 1070386.12it/s]


In [30]:
temp = [df2.loc[i].isnull().sum() for i in tqdm(df2.index)]
df2["matched"] = [1 if i < 20 else 0 for i in tqdm(temp)]
df2 = df2[df2["matched"] == 1].reset_index(drop = True)
del df2["matched"]

100%|██████████| 1309363/1309363 [00:00<00:00, 4203607.80it/s]


In [32]:
binary_columns = ["PER_WEDD_EXP", "PER_CHDB_EXP", "PER_HOUSEWF", "PER_ONEPER", "PER_PET", "CSMTC_PRFR_BIT"]
categorical_columns = ["JOB", "INCOME", "LFSTG", "TIME_CSMTC_SEG", "TIME_CAFE_SEG", "TIME_SEG"]

numeric_columns = list(df2.columns[1:-1])
for i in binary_columns:
    numeric_columns.remove(i)
for i in categorical_columns:
    numeric_columns.remove(i)

for i in numeric_columns:
    df2[i] = [np.nan if float(w) == 0 else w for w in df2[i]]

In [10]:
seg = list(seg.keys())
info_result = pd.DataFrame(index = seg, columns = binary_columns)
info_result

,PER_WEDD_EXP,PER_CHDB_EXP,PER_HOUSEWF,PER_ONEPER,PER_PET,CSMTC_PRFR_BIT
14120493,NaN,NaN,NaN,NaN,NaN,NaN
11320200,NaN,NaN,NaN,NaN,NaN,NaN
30310116,NaN,NaN,NaN,NaN,NaN,NaN
26110330,NaN,NaN,NaN,NaN,NaN,NaN
10110022,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
9310001,NaN,NaN,NaN,NaN,NaN,NaN
14110680,NaN,NaN,NaN,NaN,NaN,NaN
22320091,NaN,NaN,NaN,NaN,NaN,NaN
8210515,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# binary filter
bc = df2[binary_columns + ["SEG_KEY"]]
for i in tqdm(seg):
    temp = bc[bc["SEG_KEY"] == i].dropna().reset_index(drop = True)

    for w in binary_columns:
        if (np.mean(temp[w]) >= tolerance) & (len(temp.index) > 1):
            info_result[w].loc[i] = 1
        else:
            info_result[w].loc[i] = 0

100%|██████████| 146264/146264 [11:28<00:00, 212.38it/s]


In [12]:
# categorical filter
info_result[categorical_columns] = np.nan
cc = df2[categorical_columns + ["SEG_KEY"]]

for i in tqdm(seg):
    temp = cc[cc["SEG_KEY"] == i].dropna().reset_index(drop = True)

    for w in categorical_columns:
        try:
            if (temp[w].value_counts().iloc[0]/temp[w].value_counts().sum() >= tolerance) & (len(temp.index) > 1):
                info_result[w].loc[i] = temp[w].value_counts().index[0]
            else:
                info_result[w].loc[i] = 0
        except:
            info_result[w].loc[i] = 0

info_result

100%|██████████| 146264/146264 [09:42<00:00, 251.13it/s]


,PER_WEDD_EXP,PER_CHDB_EXP,PER_HOUSEWF,PER_ONEPER,PER_PET,CSMTC_PRFR_BIT,JOB,INCOME,LFSTG,TIME_CSMTC_SEG,TIME_CAFE_SEG,TIME_SEG
14120493,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
11320200,0,0,0,0,0,0,0.0,02_중상계층,0.0,E.18-22시,0.0,0.0
30310116,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0.0
26110330,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0.0
10110022,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9310001,0,0,0,0,0,0,0,0,0,0,0,0
14110680,0,0,0,0,0,0,0,0,0,0,0,0
22320091,0,0,0,0,0,0,0,0,0,0,0,0
8210515,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# # categorical filter
# info_result[categorical_columns] = np.nan
# cc = df2[categorical_columns + ["SEG_KEY"]]
# for i in tqdm(seg):
#     temp = cc[cc["SEG_KEY"] == i]

#     for w in categorical_columns:
#         if len(temp[w].unique()) == 1:
#             info_result[w].loc[i] = temp[w].unique()[0]
#         else:
#             info_result[w].loc[i] = 0

# info_result

In [14]:
info_result[numeric_columns] = np.nan
nc = df2[numeric_columns + ["SEG_KEY"]]

baseline = pd.DataFrame(index = numeric_columns, columns = ["mean", "std"])
for i in tqdm(numeric_columns):
    temp = df2[i].dropna().reset_index(drop = True)
    if type(temp[0]) != str:
        baseline.loc[i] = [np.mean(temp), np.std(temp)]
    else:
        baseline.loc[i] = [np.nan, np.nan]
baseline

100%|██████████| 33/33 [00:00<00:00, 140.98it/s]


,mean,std
PER_BENEFIT,4.694888,2.523435
PER_ONLINE,8.110461,5.3035
PER_ERADP,12.020185,5.42938
PER_REMM,9.419406,5.496069
PER_PRC_SNSV,7.646336,5.29535
PER_MDLV,9.008877,5.929299
PER_FRN_TRV,8.800815,5.611709
PER_DOM_TRV,8.7736,5.671169
PER_OTT,8.318192,5.490183
PER_ONGAME,10.791919,5.434051


In [15]:
if categorize == 2:
    for i in tqdm(seg):
        temp = nc[nc["SEG_KEY"] == i]
    
        for w in numeric_columns:
            if np.std(temp[w]) < baseline["std"].loc[w]:
                if np.mean(temp[w]) > baseline["mean"].loc[w]:
                    info_result[w].loc[i] = "평균 이상"
                else:
                    # info_result[w].loc[i] = "평균 이하"
                    info_result[w].loc[i] = 0
            else:
                info_result[w].loc[i] = 0

else:
    for i in tqdm(seg):
        temp = nc[nc["SEG_KEY"] == i]
    
        for w in numeric_columns:
            if np.std(temp[w]) < baseline["std"].loc[w]:
                if np.mean(temp[w]) > baseline["mean"].loc[w] + baseline["std"].loc[w]:
                    info_result[w].loc[i] = "상"
                elif np.mean(temp[w]) < baseline["mean"].loc[w] - baseline["std"].loc[w]:
                    # info_result[w].loc[i] = "하"
                    info_result[w].loc[i] = 0
                else:
                    # info_result[w].loc[i] = "중"
                    info_result[w].loc[i] = 0
            else:
                info_result[w].loc[i] = 0

100%|██████████| 146264/146264 [20:51<00:00, 116.90it/s]


In [7]:
# info_result.to_csv("information_matrix8.csv", encoding = "utf-8-sig")

In [19]:
info_result = pd.read_csv("information_matrix8.csv", encoding = "utf-8-sig", index_col = "Unnamed: 0")

def convert(val):
    if val == "0.0":
        return int(float(val))
    return val
info_result = info_result.applymap(convert)
info_result

,PER_WEDD_EXP,PER_CHDB_EXP,PER_HOUSEWF,PER_ONEPER,PER_PET,CSMTC_PRFR_BIT,JOB,INCOME,LFSTG,TIME_CSMTC_SEG,...,PER_PRC_FASHION,OTHER_CARE_AMT,PER_ORGNIC_FOOD,PER_HLTHY_FOOD,NUTRI_AMT,PER_INDR_SPRTS,ONLINE_AMT,ONLINE_RATE,PER_CAFE,PER_DRIVE
14120493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11320200,0,0,0,0,0,0,0,02_중상계층,0,E.18-22시,...,0,0,0,0,0,0,0,0,0,0
30310116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26110330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10110022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,상,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9310001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14110680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22320091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,상,0
8210515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
seg = dict(zip(df1["seg_key"], df1["BEHAVIOR_LABEL"]))

info_result["seg"] = np.nan
for i in tqdm(info_result.index):
    info_result["seg"].loc[i] = seg[i]
info_result

100%|██████████| 146264/146264 [00:31<00:00, 4660.56it/s]


,PER_WEDD_EXP,PER_CHDB_EXP,PER_HOUSEWF,PER_ONEPER,PER_PET,CSMTC_PRFR_BIT,JOB,INCOME,LFSTG,TIME_CSMTC_SEG,...,OTHER_CARE_AMT,PER_ORGNIC_FOOD,PER_HLTHY_FOOD,NUTRI_AMT,PER_INDR_SPRTS,ONLINE_AMT,ONLINE_RATE,PER_CAFE,PER_DRIVE,seg
14120493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14.0
11320200,0,0,0,0,0,0,0,02_중상계층,0,E.18-22시,...,0,0,0,0,0,0,0,0,0,11.0
30310116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30.0
26110330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26.0
10110022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,상,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9310001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.0
14110680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14.0
22320091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,상,0,22.0
8210515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.0


In [37]:
from itertools import combinations
from collections import Counter

col = info_result.columns[:-1]
result = pd.DataFrame(index = info_result.index)
result["cluster"] = np.nan

for i in tqdm(range(33)):
    temp = info_result[info_result["seg"] == i]
    lists = []
    for w in temp.index:
        lit = []
        for p in col:
            if (temp[p].loc[w] == 0) or (temp[p].loc[w] == "0"):
                pass
            else:
                lit.append(p + " " + str(temp[p].loc[w]))
        lists.append(lit)
    
    lists = dict(zip(temp.index, lists))
    
    while True:
        comb_counter = Counter()
        lits = list(lists.values())
        for s in lits:
            comb_counter.update(combinations(s, 5))
        
        most_common_combinations = comb_counter.most_common(1)
    
        if len(most_common_combinations) == 0:
            break
        
        zero = most_common_combinations[0][0][0]
        one = most_common_combinations[0][0][1]
        two = most_common_combinations[0][0][2]
        three = most_common_combinations[0][0][3]
        four = most_common_combinations[0][0][4]
        
        erase = []
        for w in lists:
            if (zero in lists[w]) and (one in lists[w]) and (two in lists[w]) and (three in lists[w]) and (four in lists[w]):
                result["cluster"].loc[w] = most_common_combinations[0][0]
                erase.append(w)
            else:
                pass
                
        for w in erase:
            del lists[w]

100%|██████████| 33/33 [04:16<00:00,  7.76s/it]


In [38]:
result.to_csv("result8.csv", encoding = "utf-8-sig")